# Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

C:\Users\20193999\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
# Input the 2019 Dataset
df = pd.read_csv('2019_data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# Display import
df.head(-5)

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,...,Context,Local Authority District code (2019),Local Authority District name (2019),Employment Domain Score,Income Domain Score,IDACI Score,IDAOPI Score,Police Strength,Police Funding,Population
0,a8977a2a4e14252420371eb993d52e4d0b8288a7c833e6...,2019-01,Metropolitan Police Service,Metropolitan Police Service,-0.709911,50.784615,On or near Rochester Way,E01031384,Arun 018E,Violence and sexual offences,...,NaN,E07000224,Arun,"0,054","0,058","0,055","0,069","30,435",2572,154.414
1,NaN,2019-01,Metropolitan Police Service,Metropolitan Police Service,0.140192,51.582311,On or near Hatch Grove,E01000027,Barking and Dagenham 001A,Anti-social behaviour,...,NaN,E09000002,Barking and Dagenham,"0,146","0,216","0,276","0,231","30,435",2572,198.294
2,934e173f2bc2e1dd3a257b37939d8f97575d3eeb89ff0c...,2019-01,Metropolitan Police Service,Metropolitan Police Service,0.140192,51.582311,On or near Hatch Grove,E01000027,Barking and Dagenham 001A,Burglary,...,NaN,E09000002,Barking and Dagenham,"0,146","0,216","0,276","0,231","30,435",2572,198.294
3,4f5b7e424bc78b1fb8c32e07da61176d2cbc5a3849d8e1...,2019-01,Metropolitan Police Service,Metropolitan Police Service,0.140634,51.583427,On or near Rams Grove,E01000027,Barking and Dagenham 001A,Burglary,...,NaN,E09000002,Barking and Dagenham,"0,146","0,216","0,276","0,231","30,435",2572,198.294
4,53d960600a4a9f54b785f598af4c75bdef2f79bce1a41b...,2019-01,Metropolitan Police Service,Metropolitan Police Service,0.141143,51.590873,On or near Furze Farm Close,E01000027,Barking and Dagenham 001A,Drugs,...,NaN,E09000002,Barking and Dagenham,"0,146","0,216","0,276","0,231","30,435",2572,198.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2348331,ca9365d6f6df6f218a06f1f32fb907061018f461bedad2...,2019-12,Hampshire Constabulary,Hampshire Constabulary,-1.068596,50.904531,On or near Carendon Close,E01023233,Winchester 014D,Violence and sexual offences,...,NaN,E07000094,Winchester,"0,047","0,055","0,068","0,082","2,697",315,119.218
2348332,091eb729242fa5503d83ae27e679b1986e48ed6da90fb0...,2019-12,Hampshire Constabulary,Hampshire Constabulary,-1.072551,50.903762,On or near Green Lane,E01023233,Winchester 014D,Violence and sexual offences,...,NaN,E07000094,Winchester,"0,047","0,055","0,068","0,082","2,697",315,119.218
2348333,0af34965867602d016c3e98b34f59c1e3c6828260ca90f...,2019-12,Hampshire Constabulary,Hampshire Constabulary,-1.074427,50.905233,On or near Glasspool,E01023233,Winchester 014D,Violence and sexual offences,...,NaN,E07000094,Winchester,"0,047","0,055","0,068","0,082","2,697",315,119.218
2348334,8e008c4280a3ef3cdfb7046444b988dbdb1f612d989fd2...,2019-12,Hampshire Constabulary,Hampshire Constabulary,-1.063798,50.902649,On or near Ashling Gardens,E01023234,Winchester 014E,Burglary,...,NaN,E07000094,Winchester,"0,064","0,091","0,093","0,142","2,697",315,119.218


In [5]:
# Drop required values that contain NAN
df.dropna(subset=['LSOA code', 'Month', 'Location', 'Crime type', 'Longitude', 'Latitude', 'Employment Domain Score',
       'Income Domain Score', 'IDACI Score', 'IDAOPI Score', 'Police Strength',
       'Police Funding', 'Population'], inplace=True)

In [9]:
df.replace(to_replace=",", value=".", regex=True, inplace=True)

In [12]:
df['Population'] = df['Population'].str.replace('.', '').fillna(df['Population']*1000)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [13]:
df['Police Strength'] = df['Police Strength'].str.replace('.','')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


# Crime Type Prediction

In [14]:
# Encode Desired Values to integers to use in models
ohe = LabelEncoder()
df['E LSOA code'] = ohe.fit_transform(df['LSOA code'].dropna().values)
df['E Month'] = ohe.fit_transform(df['Month'].dropna().values)
df['E Location'] = ohe.fit_transform(df['Location'].dropna().values)

In [15]:
# Create an X and Y
X = df[['E LSOA code', 'E Month', 'E Location', 'Longitude', 'Latitude', 'Employment Domain Score',
       'Income Domain Score', 'IDACI Score', 'IDAOPI Score', 'Police Strength',
       'Police Funding', 'Population']].values
Y = df['Crime type'].values

In [24]:
# Fit Decision Tree on Crime Types and split the dataset in training and data.
clf = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [25]:
# Predictions on Test Set
preds = clf.predict(X_test)

In [26]:
accuracy_score(y_test, preds)

0.3385904001388768

In [27]:
# Predictions in a Dataframe
df_preds = pd.DataFrame()
df_preds['y'] = y_test
df_preds['pred'] = preds

In [28]:
df_preds

,y,pred
0,Shoplifting,Other theft
1,Robbery,Violence and sexual offences
2,Violence and sexual offences,Burglary
3,Anti-social behaviour,Anti-social behaviour
4,Criminal damage and arson,Public order
...,...,...
230415,Bicycle theft,Other theft
230416,Violence and sexual offences,Public order
230417,Violence and sexual offences,Public order
230418,Shoplifting,Shoplifting


In [29]:
# Accuracy
len(df_preds.loc[df_preds['y'] == df_preds['pred']]) / len(y_test)

0.3385904001388768

In [30]:
df_preds['y'].value_counts()

Violence and sexual offences    60988
Anti-social behaviour           44148
Other theft                     21663
Vehicle crime                   20596
Criminal damage and arson       16644
Burglary                        14666
Public order                    13233
Shoplifting                     11483
Drugs                            7277
Theft from the person            6284
Robbery                          4845
Bicycle theft                    3736
Other crime                      3243
Possession of weapons            1614
Name: y, dtype: int64

In [31]:
df_preds['pred'].value_counts()

Violence and sexual offences    71117
Anti-social behaviour           48901
Other theft                     20888
Vehicle crime                   18597
Shoplifting                     14226
Criminal damage and arson       13374
Burglary                        12279
Public order                    10100
Drugs                            5924
Theft from the person            5590
Robbery                          3417
Bicycle theft                    2727
Other crime                      2243
Possession of weapons            1037
Name: pred, dtype: int64